In [ ]:
import ee 
import geemap
import os

ee.Initialize()

In [ ]:
print(os.getcwd())
india_shape = "../data/gadm/gadm36_IND_1.shp"

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import sys 
sys.path.insert(0, '../src/')
%matplotlib inline
india_gpd = gpd.read_file(india_shape)
india_gpd.plot()

In [ ]:
# Extract a specific state of india by name
import extract_gadm_data as egd

punjab_gdf = egd.extract_state_dataframe(india_gpd, 'Punjab')
punjab_gdf.plot()

In [ ]:
# Compute the extents of Punjab
punjab_extents = egd.region_extents(punjab_gdf)
print(punjab_extents)

## Create an interactive Map

In [ ]:
punjab_centre = egd.compute_center_from_extents(punjab_extents)
Map = geemap.Map(center=punjab_centre, zoom=6)
Map

In [ ]:
# Create bounding box for Punjab state and get the FIRMS collection for September 2021
punjab_bbox = egd.get_region_bounding_box(punjab_extents)
punjab_polygon = ee.Geometry.Polygon([[list(punjab_bbox[0]), list(punjab_bbox[1]), list(punjab_bbox[2]), list(punjab_bbox[3]), list(punjab_bbox[0])]])
punjab_collection = ee.ImageCollection('FIRMS').filterDate('2021-09-01', '2021-09-30').filterBounds(punjab_polygon)

In [ ]:
# Get corresponding Satellite images
punjab_satellite_collection = ee.ImageCollection('NOAA/VIIRS/001/VNP09GA').filterDate('2021-09-01', '2021-09-30').filterBounds(punjab_polygon)
rgb = punjab_satellite_collection.select(['M5', 'M4', 'M3'])
rgbVis = {'min':0.0, 'max': 3000.0}
Map.addLayer(rgb, rgbVis, 'RGB')

In [ ]:
fires = punjab_collection.select('T21')
fireVis = {'min': 300.0, 'max': 509.29, 'palette':['red', 'orange', 'yellow']}
Map.addLayer(fires, fireVis, 'Fires')

In [ ]:
# Plotting the extents polygon on the map.
Map.centerObject(punjab_polygon)
Map.addLayer(punjab_polygon, {'color': '00000000'}, 'geodesic_polygon')
punjab_planar_polygon = ee.Geometry(punjab_polygon, 0, False)
Map.addLayer(punjab_planar_polygon, {'color': '00000000'}, 'planar polygon')